In [1]:
import requests
import pandas as pd
import csv
import json
import datetime

In [2]:
### Get infromation about all the stations offered by umweltbundesamt

In [3]:
url = "https://www.umweltbundesamt.de/api/air_data/v2/stations/json?use=airquality&lang=de&date_from=2016-01-01&time_from=1&date_to=2016-10-25&time_to=24"
response = requests.get(url)
json_file = json.loads(response.content)
station_ids = json_file["data"].keys()

In [4]:
station_code = []
longitude =  []
latitude = []
for i in station_ids:
    station_code.append(json_file["data"][i][1])
    longitude.append(json_file["data"][i][7])
    latitude.append(json_file["data"][i][8]) 

In [5]:
df = pd.DataFrame(zip(station_ids,station_code,longitude,latitude),columns=["Id","Station","Longitude","Latitude"])
# df.to_csv('stations_air_quality.csv',index=False)

In [7]:
### Now get air quality information from all those weather stations for the desired time

In [8]:
# !mkdir data_air_quality

In [9]:
for i in station_ids:
    try:
        date_ranges = ["date_from=2019-10-25&time_from=24&date_to=2020-10-25&time_to=23",
                       "date_from=2018-10-25&time_from=24&date_to=2019-10-25&time_to=23",
                       "date_from=2017-10-25&time_from=24&date_to=2018-10-25&time_to=23",
                       "date_from=2016-10-25&time_from=24&date_to=2017-10-25&time_to=23",
                       "date_from=2016-01-01&time_from=1&date_to=2016-10-25&time_from=23"]
        for information_range in date_ranges:
        #     information_range = "date_from=2016-01-01&time_from=1&date_to=2017-01-01&time_to=24"
            station = "&station=" + i

            #load data
            url = "https://www.umweltbundesamt.de/api/air_data/v2/airquality/csv?"+information_range+station+"&lang=de"
            response = requests.get(url)
            decode = response.content.decode('utf-8')
            cr = csv.reader(decode.splitlines(), delimiter=';')

            # turn into dataframe and do some formatting
            df = pd.DataFrame(list(cr))
            new_header = df.iloc[0] 
            df = df[1:-2] 
            df.columns = new_header 

            select = df["Datum"].str.contains('24:00')
            df2 = df.loc[select,"Datum"]
            for row,value in enumerate(df2):
                dtf = datetime.datetime.strptime((value[1:-6]+"23:00"), '%d.%m.%Y %H:%M')
                dtf += datetime.timedelta(hours=1)
                df2.iloc[row] = "\'" + datetime.datetime.strftime(dtf , '%d.%m.%Y %H:%M') + "\'"
            df.loc[select,"Datum"] = df2
            df['Datum'] =  pd.to_datetime(df['Datum'], format = "'%d.%m.%Y %H:%M'" )

            station_name = df.loc[1,"﻿Stationscode"] 

            df = df.rename(columns={"Datum": "Date", 
                                    "Feinstaub (PM₁₀) stündlich gleitendes Tagesmittel in µg/m³": (station_name +" Particulate matter in µg/m³"),
                                    "Ozon (O₃) Ein-Stunden-Mittelwert in µg/m³":(station_name + " Ozone in µg/m³"),
                                    'Stickstoffdioxid (NO₂) Ein-Stunden-Mittelwert in µg/m³': (station_name+" NO₂ in µg/m³")})

            df = df.drop(["Luftqualitätsindex"], axis=1)
            df = df.drop(["﻿Stationscode"], axis=1)
            path = "./data_air_quality/" + station + information_range + ".csv"
            df.to_csv(path,index=False)
    except:
        print("Response =",response.status_code)
        print("Did not load data for station", i)



Response = 500
Did not load data for station 85
Response = 500
Did not load data for station 97
Response = 500
Did not load data for station 264
Response = 500
Did not load data for station 686
Response = 500
Did not load data for station 797
Response = 500
Did not load data for station 825
Response = 500
Did not load data for station 850
Response = 500
Did not load data for station 876
Response = 500
Did not load data for station 953
Response = 500
Did not load data for station 1145
Response = 500
Did not load data for station 1449
Response = 500
Did not load data for station 1515
Response = 500
Did not load data for station 1533
Response = 500
Did not load data for station 1555
Response = 500
Did not load data for station 1560
Response = 500
Did not load data for station 1599
Response = 500
Did not load data for station 1767
Response = 500
Did not load data for station 1878
Response = 500
Did not load data for station 1988


In [10]:
## in case we want to download data differently:
# url = "https://www.umweltbundesamt.de/api/air_data/v2/measures/csv?date_from=2017-08-01&time_from=12&date_to=2017-08-01&time_to=12&data%5B0%5D%5Bco%5D=3&data%5B0%5D%5Bsc%5D=4&lang=de"
# url = "https://www.umweltbundesamt.de/api/air_data/v2/airquality/csv?date_from=2016-01-01&time_from=1&date_to=2017-01-01&time_to=24&station=1122&lang=de"